# 7.3 String Manipulation

python很多内建方法很适合处理string。而且对于更复杂的模式，可以配合使用正则表达式。而pandas则混合了两种方式。

# 1 String Object Methods

大部分string处理，使用内建的一些方法就足够了。比如可以用split来分割用逗号区分的字符串：

In [44]:
val = 'a,b, guido'

In [45]:
val.split(',')

['a', 'b', ' guido']

In [46]:
pieces = [x.strip() for x in val.split(',')]
pieces

['a', 'b', 'guido']

In [47]:
first, second, third = pieces

In [48]:
first + '::' + second + '::' + third

'a::b::guido'

In [49]:
'::'.join(pieces)

'a::b::guido'

其他一些方法适合锁定子字符串位置相关的。用in关键字是检测substring最好的方法，当然，index和find也能完成任务：

In [50]:
'guido' in val

True

In [51]:
val.index(',')

1

In [52]:
val.find(':')

-1

In [53]:
val.count(',')

2

In [54]:
val.replace(',', '::')

'a::b:: guido'

In [55]:
val.replace(',', '')

'ab guido'



# 2 Regular Expressions（正则表达式）



re模块有以下三个类别：patther matching（模式匹配）, substitution（替换）, splitting（分割）。通常这三种都是相关的，一个regex用来描述一种pattern，这样会有很多种用法。这里举个例子，假设我们想要根据空格（tabs，spaces，newlines）来分割一个字符串。用于描述一个或多个空格的regex是`\s+`:

In [56]:
import re

In [57]:
text = "foo    bar\t baz  \tqux"

In [58]:
re.split('\s+', text)

['foo', 'bar', 'baz', 'qux']

In [59]:
regex = re.compile('\s+')

In [60]:
regex.split(text)

['foo', 'bar', 'baz', 'qux']

In [61]:
regex.findall(text)

['    ', '\t ', '  \t']

In [62]:
text = """Dave dave@google.com 
          Steve steve@gmail.com 
          Rob rob@gmail.com 
          Ryan ryan@yahoo.com """

pattern = r'[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}'

In [63]:
# re.IGNORECASE makes the regex case-insensitive 
regex = re.compile(pattern, flags=re.IGNORECASE)

使用findall找到一组邮件地址：

In [64]:
regex.findall(text)

['dave@google.com', 'steve@gmail.com', 'rob@gmail.com', 'ryan@yahoo.com']

In [65]:
m = regex.search(text)

In [66]:
m

<_sre.SRE_Match object; span=(5, 20), match='dave@google.com'>

In [67]:
text[m.start():m.end()]

'dave@google.com'

regex.match返回None，因为它只会在pattern存在于stirng开头的情况下才会返回匹配结果：

In [68]:
print(regex.match(text))

None


而sub返回一个新的string，把pattern出现的地方替换为我们指定的string：

In [69]:
print(regex.sub('REDACTED', text))

Dave REDACTED 
          Steve REDACTED 
          Rob REDACTED 
          Ryan REDACTED 


In [70]:
pattern = r'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})'

In [71]:
regex = re.compile(pattern, flags=re.IGNORECASE)

In [72]:
m = regex.match('wesm@bright.net')

In [73]:
m.groups()

('wesm', 'bright', 'net')

In [74]:
regex.findall(text)

[('dave', 'google', 'com'),
 ('steve', 'gmail', 'com'),
 ('rob', 'gmail', 'com'),
 ('ryan', 'yahoo', 'com')]

sub也能访问groups的结果，不过要使用特殊符号 \1, \2。\1表示第一个匹配的group，\2表示第二个匹配的group，以此类推：

In [75]:
print(regex.sub(r'Username: \1, Domain: \2, Suffix: \3', text))

Dave Username: dave, Domain: google, Suffix: com 
          Steve Username: steve, Domain: gmail, Suffix: com 
          Rob Username: rob, Domain: gmail, Suffix: com 
          Ryan Username: ryan, Domain: yahoo, Suffix: com 


这里给一些正则表达式的方法：

![](http://oydgk2hgw.bkt.clouddn.com/pydata-book/mj4vc.png)

# 3 Vectorized String Functions in pandas（pandas中的字符串向量化函数）

一些复杂的数据清理中，string会有缺失值：

In [76]:
import numpy as np
import pandas as pd

In [77]:
data = {'Dave': 'dave@google.com', 'Steve': 'steve@gmail.com', 
        'Rob': 'rob@gmail.com', 'Wes': np.nan}

In [78]:
data = pd.Series(data)
data

Dave     dave@google.com
Steve    steve@gmail.com
Rob        rob@gmail.com
Wes                  NaN
dtype: object

In [79]:
data.isnull()

Dave     False
Steve    False
Rob      False
Wes       True
dtype: bool

In [80]:
data.str

In [81]:
data.str.contains('gmail')

Dave     False
Steve     True
Rob       True
Wes        NaN
dtype: object

In [82]:
pattern

'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\\.([A-Z]{2,4})'

In [83]:
matches = data.str.match(pattern, flags=re.IGNORECASE)
matches

Dave     True
Steve    True
Rob      True
Wes       NaN
dtype: object

In [84]:
matches.str.get(1)

Dave    NaN
Steve   NaN
Rob     NaN
Wes     NaN
dtype: float64

In [85]:
matches.str.get(0)

Dave    NaN
Steve   NaN
Rob     NaN
Wes     NaN
dtype: float64

In [86]:
data.str[:5]

Dave     dave@
Steve    steve
Rob      rob@g
Wes        NaN
dtype: object